In [20]:
import requests
from pywikibot.data import api
import pywikibot
import pprint
import re 
import requests

from pywikibot import pagegenerators

In [21]:
def check_url(qurl):

    request = requests.get("https://www.theguardian.com/" + qurl)
    if request.status_code == 200:
        return qurl
    else:
        return None
        
    return None

In [22]:
def qid_claim(page, text):

    
    qid_check=re.findall(r'[*]+\s*[{{]+\s*[G]+[u]+[a]+[r]+[d]+[i]+[a]+[n]+\s*[t]+[o]+[p]+[i]+[c]+\s*[|]+[^\n]*',text)
    
    qid_page=[]  
    try:
        
        for x in qid_check:
            qid_check=x.split("|")[1].replace("}","")
            
            qid_page.append(check_url(qid_check))   

    except:
        pass
    
    return qid_page

In [23]:
def get_claim(item, prop):
    """
    Requires a property, item, and list of qualifier properties and items.
    Returns claim that matches or None.
    """
    
    item_dict = item.get()
    
    claim_s=[]
    claims_url=[]
    
    try:
        item_dict['claims'][prop]

    except:
        return None

    for claim in item_dict['claims'][prop]:
        if not claim.getSnakType() == 'somevalue':
            #print(type(claim))
            claim_s.append(claim)
            print(claim.getTarget())
            claims_url.append(claim.getTarget())
        else:
            pass   
    return claim_s, claims_url

In [24]:
def addclaim(query,cat, qid_page):


    
    site = pywikibot.Site("en", "wikipedia")
    repo = site.data_repository()

#Firstly I will make a page object using the title , then using the data_item() method I will connect to the wikidata item and edit
    
    for title_value in range(len(query)):
        page = pywikibot.Page(site, query[title_value])
        item=page.data_item()
        
        print("Title of page:", query[title_value])
        
        target_id=qid_page[query[title_value]]
        
        print("Url against Wikipedia page with Guardian Topic Id, None represents invalid url encountered", target_id,'\n')
        
        print("Existing claim ID")
        
        pre_claims_url=[]
        
        try:
            pre_claim, pre_claims_url=get_claim(item,cat)
        except:
            pass
        
        diff=set(target_id)-set(pre_claims_url)
        
        print("Url to be added", diff,'\n')
        
        if len(diff)!=0:
            for tar in diff:
                if tar!=None:
                    Identifiersclaim = pywikibot.Claim(repo, cat)
                    Identifiersclaim.setTarget(tar)
                    item.addClaim(Identifiersclaim, summary=u'Adding Guardian ID') 

        else:
            pass
        
    
    return None 

In [25]:
def cate(category):
    
    site = pywikibot.Site('en', 'wikipedia')
    repo = site.data_repository() 
    #Creating an object of the Category class of the category to use the methods defined on the class

    cat = pywikibot.Category(site, category)
    property_value_cat=cat.templatesWithParams()[0][1][0].split("=")[1]
    #This is the property that is to be checked if present or not, else to be added to all the pages. 

    print("Property Value:",property_value_cat)
    
    #To generate an iterator for the pages in the category

    gen = pywikibot.pagegenerators.CategorizedPageGenerator(cat)
    
    query=[]
    
    for page in gen:
        query.append(page.title())
    
    #print("Pages inside the category", query)
    
    #This list will consist of all the url links which are to be set as target for the ID claim
    qid_page={}

    for page_title in query:
        page = pywikibot.Page(site, page_title)
        text=page.text
        qid_page[page_title]=qid_claim(page, text)
        
    #print(qid_page)
    
    addclaim(query,property_value_cat, qid_page)
    
    return None

In [7]:
#references are not shown in page.text and hence some times it is not working 

In [8]:
#https://bitbucket.org/mikepeel/wikicode/src/master/enwp_find_wikidata.py

category="Guardian topic ID different from Wikidata"

cate(category)

Property Value: P3106
Title of page: Arab Spring
Url against Wikipedia page with Guardian Topic Id, None represents invalid url encountered ['world/arab-and-middle-east-protests', 'world/interactive/2011/mar/22/middle-east-protest-interactive-timeline'] 

Existing claim ID
world/arab-and-middle-east-protests
Url to be added {'world/interactive/2011/mar/22/middle-east-protest-interactive-timeline'} 



Sleeping for 9.6 seconds, 2021-05-06 18:28:16


Title of page: Jimi Hendrix
Url against Wikipedia page with Guardian Topic Id, None represents invalid url encountered [None] 

Existing claim ID
music/jimi-hendrix
Url to be added {None} 



In [9]:
#Re-run to check the changes 

category="Guardian topic ID different from Wikidata"

cate(category)

Property Value: P3106
Title of page: Arab Spring
Url against Wikipedia page with Guardian Topic Id, None represents invalid url encountered ['world/arab-and-middle-east-protests', 'world/interactive/2011/mar/22/middle-east-protest-interactive-timeline'] 

Existing claim ID
world/arab-and-middle-east-protests
world/interactive/2011/mar/22/middle-east-protest-interactive-timeline
Url to be added set() 

Title of page: Jimi Hendrix
Url against Wikipedia page with Guardian Topic Id, None represents invalid url encountered [None] 

Existing claim ID
music/jimi-hendrix
Url to be added {None} 



In [17]:
category="Guardian topic ID not in Wikidata"

cate(category)

Property Value: P3106
Title of page: Arab Spring concurrent incidents
Url against Wikipedia page with Guardian Topic Id, None represents invalid url encountered ['world/arab-and-middle-east-protests', 'world/interactive/2011/mar/22/middle-east-protest-interactive-timeline'] 

Existing claim ID
Url to be added {'world/interactive/2011/mar/22/middle-east-protest-interactive-timeline', 'world/arab-and-middle-east-protests'} 



Sleeping for 19.5 seconds, 2021-05-06 18:37:59
